# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице 12 столбцов:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Наименование столбцов соответствует стилю. Столбцы имеют различный тип данных.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В столбце **education** данные с разными регистрами.

Предпологаяю, что столбец **days_employed** выражен в формате Unix time. Этот столбец не нужен для вычисления влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Поэтому им можно принибречь.




## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# Код ревьюера

df_copy = df.copy()

In [6]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
income_median = df['total_income']. median()
df['total_income'] = df['total_income'].fillna(value = income_median)

In [49]:
# Код ревьюера

display(df_copy[df_copy['total_income'].isna()]['income_type'].value_counts(normalize=True))
display(df_copy[df_copy['total_income'].notna()]['income_type'].value_counts(normalize=True))

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

сотрудник          0.517493
компаньон          0.236525
пенсионер          0.177924
госслужащий        0.067800
безработный        0.000103
в декрете          0.000052
предприниматель    0.000052
студент            0.000052
Name: income_type, dtype: float64

In [9]:
days_employed_median = df['days_employed']. median()
df['days_employed'] = df['days_employed'].fillna(value = days_employed_median)

In [10]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [11]:
df ['debt'].unique()

array([0, 1])

In [12]:
df ['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [13]:
df ['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [14]:
df ['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [15]:
df ['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

Как было сказано ранне столбцом 'days_employed' можно принебречь. Я заполнил пропуски в этом столбце медианным значением.
Данные в столбце total_income нужны для того, чтобы определить есть ли зависимость между уровнем дохода и возвратом кредита в срок. Доля пропущенных значение в этом столбце составляет 10%. Есть несколько вариантов как можно обработать эти пропуски
1) заменить медианным значением
2) заменить средним арифметическим занчение
3) удалить строки где есть пропущенные значения в этом столбце.

Я выбрал первый вариант.

В столбце **'children'** есть значения -1 или 20. Это могут быть ошибочные значения связанные с ошибочным заполнение данных клиентом. Значения - -1 и 20 состаявлют 0,5 % от всех данных. Поэтому считаю, что эти данные не повлияют на определение зависимости между наличием детей и возвратом кредита в срок.

### Замена типа данных

In [16]:
df ['total_income'] = df ['total_income'].astype('int')

In [17]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Для удобства расчетов, тип данных столбца **total_income** был заменен на **int**. Был использован метод astype().

### Обработка дубликатов

In [18]:
df['education']= df['education'].str.lower()

In [19]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [20]:
df ['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [21]:
df ['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [22]:
df ['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Вывод**

В даннном пункте, в столбце **education** были обработаны дубликаты, которые были в разных регистрах. Причина появление данных в разных регистрах - заполнение клиентов данных разным шрифтом.
Данные в столбце были приведены к ниженму регистру с помощью команды str.lower(). 
Для поиска дубликатов используется метод value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке. Применяют метод к объекту Series. Результат его работы – список пар «значение-частота», отсортированные по убыванию. Все дубликаты, которые встречаются чаще других, оказываются в начале списка.

В столбце **purpose** нужно с помощью лемматизации объеденить  цели в одну. Например, "свадьба" и "проведение свадьбы" в цель - "свадьба"

### Лемматизация

In [23]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
def lemma(purpose):
    lemma_cell = ' ' .join(m.lemmatize(purpose))
    purpose_lem_arr = ["жилье", "свадьба", 'автомобиль',"образование","недвижимость","строительство"]
    for purpose in purpose_lem_arr:
        if purpose in lemma_cell:
            return purpose
 
df['purpose_lem'] = df['purpose'].apply(lemma)
df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [24]:
df ['purpose_lem'].value_counts()

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_lem, dtype: int64

In [25]:
df['purpose_lem'].value_counts(dropna=False)

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_lem, dtype: int64

**Вывод**

В данном пункте был лемматизирован столбец **purpose**  и добавлен новый столбец **purpose_lem** с такими леммами как "жилье", "свадьба", 'автомобиль',"образование","недвижимость","строительство".

### Категоризация данных

In [26]:
def category(row):
    if 'недвижимос' in row:
        return 'недвижимость'
    if 'жиль' in row:
         return 'недвижимость'
    if 'свадьб' in row:
        return 'свадьба'
    if 'автомоб' in row:
        return 'автомобиль'
    if 'образован' in row:
        return 'образование'

df ['purpose'] = df['purpose'].apply(category)


In [27]:
df ['purpose'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose, dtype: int64

**Или категоризацию можно применить к столбцу с леммами**

In [28]:
def category(row):
    if 'жиль' in row:
         return 'недвижимость'
    if 'недвижимос' in row:
        return 'недвижимость'
    if 'свадьб' in row:
        return 'свадьба'
    if 'автомоб' in row:
        return 'автомобиль'
    if 'образован' in row:
        return 'образование'
df ['purpose_lem'] = df['purpose_lem'].apply(category)


In [29]:
df['purpose_lem'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_lem, dtype: int64

In [30]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
purpose_lem         0
dtype: int64

In [31]:
pd.qcut(df['total_income'], q=3).value_counts()

(170468.0, 2265604.0]      7175
(123082.667, 170468.0]     7175
(20666.999, 123082.667]    7175
Name: total_income, dtype: int64

In [32]:
def category(income):
    if 20666 < income < 123083:
        return 'низкий доход'
    if 123083 <= income < 170468:
        return 'средний доход'
    if income >= 170468:
        return 'высокий доход'

df['total_income'] = df['total_income'].apply(category)

In [33]:
df['total_income'].value_counts()

низкий доход     7175
средний доход    7175
высокий доход    7175
Name: total_income, dtype: int64

In [34]:
def category(child):
    if -1 < child <= 0:
        return 'детей нет'
    if child > 0:
        return 'есть дети'

df['children'] = df['children'].apply(category)
df['children'].value_counts(dropna=False)

детей нет    14149
есть дети     7329
NaN             47
Name: children, dtype: int64

In [35]:
def category(debt):
    if  debt == 0:
        return 'долга нет'
    if debt == 1:
        return 'долг есть'

df['debt'] = df['debt'].apply(category)
df['debt'].value_counts(dropna=False)

долга нет    19784
долг есть     1741
Name: debt, dtype: int64

**Вывод**

В пункте были категоризованы столбцы:
- purpose - столбец был категоризован на 3 группы : "низкий", "средний", и "высокий доход".
- family_status_id - столбец был категоризован на 2 группы: "в браке" и "не в браке"
- debt - столбец был категоризован на 2 группы: "долг есть" и "нет долга"
- children - столбец был категоризован на 2 групп: "есть дети" и "детей нет"

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [36]:
children_pivot  = df.pivot_table(index = ['children'], columns = ['debt'],
                                values = ['purpose'], aggfunc = 'count')
children_pivot

purpose          
debt      долг есть долга нет
children                     
детей нет      1063     13086
есть дети       677      6652

In [37]:
children_pivot ['ratio'] = children_pivot['purpose']['долг есть'] / (children_pivot['purpose']['долг есть'] + children_pivot['purpose']['долга нет'])
children_pivot

purpose               ratio
debt      долг есть долга нет          
children                               
детей нет      1063     13086  0.075129
есть дети       677      6652  0.092373

**Вывод**

Доля клиентов  с детьми и у которых был долг составила 9.2%. А у клиентов у кого не было детей составила 7.5%.
Это говорит о том, что есть небольшая зависимость между наличием детей и возвратом кредита в срок.

**p.s**
Как перевести столбец ratio в вид %?

In [38]:
# Код ревьюера
with pd.option_context('display.float_format', '{:.2%}'.format):
    display(children_pivot)

purpose           ratio
debt      долг есть долга нет      
children                           
детей нет      1063     13086 7.51%
есть дети       677      6652 9.24%

In [39]:
df.groupby('children').apply(lambda group : pd.Series(
                dict(число_клиентов=group['debt'].count(), 
                     должников=group[group['debt'] == 'долг есть']['debt'].count(), 
                     доля_должников=(group['debt']=='долг есть').mean()))).astype({'число_клиентов':int,'должников':int})

,число_клиентов,должников,доля_должников
children,,,
детей нет,14149,1063,0.075129
есть дети,7329,677,0.092373


- Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [40]:
family_pivot = df.pivot_table(index = ['family_status'], columns = ['debt'],
values = ['purpose'], aggfunc = 'count')
family_pivot

purpose          
debt                  долг есть долга нет
family_status                            
Не женат / не замужем       274      2539
в разводе                    85      1110
вдовец / вдова               63       897
гражданский брак            388      3789
женат / замужем             931     11449

In [41]:
family_pivot ['ratio'] = family_pivot['purpose']['долг есть'] / (family_pivot['purpose']['долг есть'] + family_pivot['purpose']['долга нет'])
family_pivot.sort_values(by = 'ratio',ascending = False)

purpose               ratio
debt                  долг есть долга нет          
family_status                                      
Не женат / не замужем       274      2539  0.097405
гражданский брак            388      3789  0.092890
женат / замужем             931     11449  0.075202
в разводе                    85      1110  0.071130
вдовец / вдова               63       897  0.065625

In [51]:
with pd.option_context('display.float_format', '{:.2%}'.format):
    display(family_pivot.sort_values(by='ratio'))

purpose           ratio
debt                  долг есть долга нет      
family_status                                  
вдовец / вдова               63       897 6.56%
в разводе                    85      1110 7.11%
женат / замужем             931     11449 7.52%
гражданский брак            388      3789 9.29%
Не женат / не замужем       274      2539 9.74%

**Вывод**

Зависимость есть. Люди, которые не были в браке или люди, у которых брак не зарегистирован имеет большую склонность иметь задолженность по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
income_pivot = df.pivot_table(index = ['total_income'], columns = ['debt'],
values = ['family_status_id'], aggfunc = 'count')
income_pivot

family_status_id          
debt                 долг есть долга нет
total_income                            
высокий доход              543      6632
низкий доход               583      6592
средний доход              615      6560

In [44]:
income_pivot ['ratio'] = income_pivot['family_status_id']['долг есть'] / (income_pivot['family_status_id']['долг есть'] + income_pivot['family_status_id']['долга нет'])
with pd.option_context('display.float_format', '{:.2%}'.format):
    display(income_pivot)

family_status_id           ratio
debt                 долг есть долга нет      
total_income                                  
высокий доход              543      6632 7.57%
низкий доход               583      6592 8.13%
средний доход              615      6560 8.57%

In [45]:
income_pivot.sort_values(by = 'ratio',ascending = False)
income_pivot

family_status_id               ratio
debt                 долг есть долга нет          
total_income                                      
высокий доход              543      6632  0.075679
низкий доход               583      6592  0.081254
средний доход              615      6560  0.085714

**Вывод**

Зависимость есть. Самая низкая доля клиентов с высоким доходом, у которых есть долг.
Но доля клиентов со средним доходм и низким дохом, у которыл был долг - примерно одинакова.

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
purpose_pivot = df.pivot_table(index = ['purpose'], columns = ['debt'],
values = ['family_status_id'], aggfunc = 'count')
purpose_pivot ['ratio'] = purpose_pivot['family_status_id']['долг есть'] / (purpose_pivot['family_status_id']['долг есть'] + purpose_pivot['family_status_id']['долга нет'])
with pd.option_context('display.float_format', '{:.2%}'.format):
    display(purpose_pivot)


family_status_id           ratio
debt                долг есть долга нет      
purpose                                      
автомобиль                403      3912 9.34%
недвижимость              782     10058 7.21%
образование               370      3652 9.20%
свадьба                   186      2162 7.92%

In [47]:
purpose_pivot.sort_values(by = 'ratio',ascending = False)

family_status_id               ratio
debt                долг есть долга нет          
purpose                                          
автомобиль                403      3912  0.093395
образование               370      3652  0.091994
свадьба                   186      2162  0.079216
недвижимость              782     10058  0.072140

**Вывод**

Самая большая доля клиентов, у которых был долг брали кредит на образование. Клиенты, которые брали кредит на недвижимость, меньше по сравнению с остальными целями имели долг.

## Шаг 4. Общий вывод

Была проведена работа по предобработке данных в результате, которой было выяснено, что наличие детей, цели кредиты, уровень дохода могут влиять на возврат кредита. Но опираться только на эти данные для оценки способности потенциального заёмщика вернуть кредит банку не стоит. Так как доля клиентов в зависимости от наличия детей, целей кредита и уровеня дохода , которые не возвращают кредит, меньше 10%.